In [1]:
import numpy as np
import os
import scipy.stats as stats
import scipy.io as io
import networkx as nx
import random
import itertools
import glob
from scipy.stats import linregress
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix,roc_curve,roc_auc_score
from xgboost import XGBClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import seaborn as sns
from scipy.io import loadmat, savemat
#from fracModel import fracOrdUU
from collections import Counter
from collections import defaultdict
from numpy import arange,array,ones,linalg
from sklearn.linear_model import LinearRegression
import pandas as pd
import powerlaw
import os.path
from os import path
import warnings
warnings.filterwarnings("ignore")
import pickle

In [2]:
##Calculate entropy
def Entropy_MC(s,b,k,beta,gamma):
    #s = s
    inv, l, n = k-b, len(s), 4**k
    T=np.zeros((n,n))
    count = [0]*n
    word_list = [''.join(x) for x in itertools.product('acgt', repeat=k)]
    wd = dict(zip(word_list, list(range(len(word_list)))))
    uu = []
    for i in range(k,l-b):
        n1, n2 = wd[s[i-k:i]], wd[s[i-k+b:i+b]]
        T[n1,n2] += 1
        count[n1] += 1
        
    wo_next = [''.join(x) for x in itertools.product('acgt', repeat=b)]
    for i in range(n):
        wo = word_list[i]
        for l in wo_next:
            j = wd[wo[b:]+l]
            T[i,j] = (T[i,j]+beta)/(count[i]+4**b*beta)
    sum_count = np.sum(count)+4**k*gamma
    
    prob = [(count[i]+gamma)/sum_count for i in range(len(count))] 
    n = len(T)
    H = 0
    F = []
    for i in range(n):
        wo = word_list[i]
        for l in wo_next:
            j = wd[wo[b:]+l]
            F.append(T[i,j])
            H-=prob[i]*T[i,j]*np.log2(T[i,j])
    return F,H

In [23]:
from collections import defaultdict
def getSeqs(filename):
    seq_list = defaultdict(list)
    mapseq_list = defaultdict(list)
    mapping = {'A': 'a', 'T': 't', 'C': 'c', 'G': 'g','a': 'a', 't': 't', 'c': 'c', 'g': 'g'}
    with open(filename) as f:
        j = -1
        for i, line in enumerate(f):
            if line.startswith('>'):
                j += 1
            else:
                this_line = list(line)
                this_line = list(filter(lambda ch: ch in 'acgtACGT', this_line))
                seq_list[j].extend(this_line)
                mapseq_list[j] = ''.join(list(map(lambda ch: mapping[ch], seq_list[j])))
    return mapseq_list

In [24]:
def stats(s,b,k,beta,gamma):
    F_list = []
    en_list=[]
    for i in range(len(s)):
        F_1,en = Entropy_MC(s[i],b,k,beta,gamma) #Transition Probability vector
        F = F_1
        F_list.append(F)
        en_list.append(en)
    return F_list,en_list

In [ ]:
status = [['20A'],['19A'],['20B'],['19B'],['20C']]
V = []
for stat in status:
    filenames = []
    for s in stat:
        filenames.append('Clade_Analysis/'+s+'.fasta')
    print(filenames)
    sequences = []
    for filename in filenames:
        sequences.append(getSeqs(filename))  
    V.append(sequences)

In [ ]:
with open('Clade_Analysis/V_clades.pkl','wb') as f:  
    pickle.dump(V,f)

In [25]:
def conv_index(b,k,ind):
    word_list = [''.join(x) for x in itertools.product('acgt', repeat=k)]
    wd_next = [''.join(x) for x in itertools.product('acgt', repeat=b)]
    l = 4**b
    rem = ind%l
    st =' -> '+wd_next[rem]
    num = int(ind/l)
    st = word_list[num]+st
    return st
def normalize(x):
        return (x - x.min(0)) / x.ptp(0)

In [32]:
def ML(b,k,V,beta,gamma,classes):
    word_list = [''.join(x) for x in itertools.product('acgt', repeat=k)]
    wd_next = [''.join(x) for x in itertools.product('acgt', repeat=b)]
    X = []
    y = []
    ct = 0
    n = len(V)
    G = [[] for i in range(n)]
    
    for seq in V:
        for s in seq:
            F_list,en_list = stats(s,b,k,beta,gamma)
            G[ct]+=F_list
        print(len(G[ct]))
        ct+=1
    
    
    min_val = float('inf')
    for i in range(n):
        if min_val > len(G[i]):
            min_val = len(G[i])
    #Histograms
    
    for i in range(n):
        random.shuffle(G[i])
        X+=[G[i][j] for j in range(len(G[i]))]
        y+=[i for j in range(len(G[i]))]
    #min_val = int(3*(min_val)/4)   
    print('Length of X: {}'.format(len(X)))
    X_original = [x_i for x_i in X]
    X = np.array(X)
    y = np.array(y)
    X_original = np.array(X_original)
    X = preprocessing.scale(X)
    U_X = list(X[:min_val])
    U_y = list(y[:min_val])
    T_X = list(X[min_val:len(G[0])])
    T_y = list(y[min_val:len(G[0])])
    running_sum = len(G[0])
    for i in range(1,n):
        U_X+=list(X[running_sum:running_sum+min_val])
        U_y+=list(y[running_sum:running_sum+min_val])
        T_X+=list(X[running_sum+min_val:running_sum+len(G[i])])
        T_y+=list(y[running_sum+min_val:running_sum+len(G[i])])
        running_sum+=len(G[i])
    zipped_U = list(zip(U_X,U_y))
    random.shuffle(zipped_U)
    U_X = [x_i for x_i,_ in zipped_U]
    U_y = [y_i for _,y_i in zipped_U]
    zipped_T = list(zip(T_X,T_y))
    random.shuffle(zipped_T)
    T_X = [x_i for x_i,_ in zipped_T]
    T_y = [y_i for _,y_i in zipped_T]
    U_X = np.array(U_X)
    U_y = np.array(U_y)
    T_X = np.array(T_X)
    T_y = np.array(T_y)
    return U_X,U_y,X_original,T_X,T_y
def build_model(X,y,X_original,b,k,rr,cutoff,T,y_T,f):    #Building model for feature selection
    kf = KFold(n_splits = 4)
    Acc = []
    Acc_T = []
    sc_prob_T = []
    n = len(rr)
    val_score = [[] for i in range(n)]
    feat_set = set()
    feat_set_ind = set()
    D = {}
    X_orig_mean = np.mean(X_original,axis = 0)
    X_orig_std = np.std(X_original,axis = 0)
    fig, ax = plt.subplots()
    gg = 0
    mean_fpr = np.linspace(0, 1, 100)
    tprs = []
    aucs = []
    prec = []
    rec = []
    n_dim = 4**(k+b) 
    acc_list = []
    plt.rcParams.update({'font.size': 14})
    for train, test in kf.split(X):
        X_train = X[train]
        y_train = y[train]
        X_test = X[test]
        y_test = y[test]
        model = XGBClassifier(max_depth=1)
        model.fit(X_train, y_train)
        imp_feat = model.feature_importances_
        imp_indices = np.argsort(imp_feat)
        for im in range(cutoff,0,1):
            transit = conv_index(b,k,imp_indices[im])
            D[transit]  = (round(X_orig_mean[imp_indices[im]],2),round(X_orig_std[imp_indices[im]],4))
            feat_set.add(transit)
            feat_set_ind.add(imp_indices[im])
        #plt.bar(range(len(imp_feat)),imp_feat)
        #plt.show()
        #plt.close()
        imp_feat.sort()
        #feature selection and rebuilding the model using selected features
        thresh = imp_feat[cutoff]
        selection = SelectFromModel(model, threshold=thresh, prefit=True)
        select_X_train = selection.transform(X_train)
        #selection_model = XGBClassifier(max_depth = 1)
        selection_model = LogisticRegression()
        selection_model.fit(select_X_train, y_train)
        select_X_test = selection.transform(X_test)
        scores = selection_model.predict(select_X_test)
        scores_prob = selection_model.predict_proba(select_X_test)
        if len(T) > 0:
            select_T = selection.transform(T)
            scores_T = selection_model.predict(select_T)
            scores_prob_T = selection_model.predict_proba(select_T)
            Acc_T.append(accuracy_score(scores_T,y_T))
            sc_prob_T+=list(scores_prob_T)
        ctr = 0
        for yy in y_test:
            val_score[yy].append(scores_prob[ctr])
            ctr+=1
        ctr = 0
        val_score_dis = [0 for i in range(n)]
        ctx = [0 for i in range(n)]
        for yy in y_test:
            if scores[ctr] == yy:
                val_score_dis[yy]+=1
            ctx[yy]+=1
            ctr+=1
        for yc in range(n):
            val_score_dis[yc] = val_score_dis[yc]/ctx[yc]*100
        acc_list.append(val_score_dis)
        imp_feat_main = selection_model.coef_
        
        Acc.append(accuracy_score(scores,y_test))
        
        gg+=1
    Acc = np.array(Acc)
    val_score = np.array(val_score)
    if len(T)> 0:
        Acc_T = np.array(Acc_T)
    plt.rcParams.update({'font.size': 18})
    acc_list_mean = np.mean(acc_list,axis =0)
    acc_list_std = np.std(acc_list,axis =0)
    #f.write("Mean Accuracy: "+str(acc_list_mean)+'\n')
    #f.write("Std Accuracy: "+str(acc_list_std)+'\n')
    #print("Mean Accuracy: {}".format(acc_list_mean))
    #print("Std Accuracy: {}".format(acc_list_std))
    for i in range(n):
        probs_mean = np.mean(val_score[i],axis = 0)
        probs_std = np.std(val_score[i],axis = 0)
        #print(probs_mean)
        #print('{}'.format(rr[i]))
        #f.write(str(i)+':'+str(probs_mean)+' '+str(probs_std))
        #f.write('\n')
        plt.bar(rr,probs_mean,yerr=probs_std, align='center', color=['red', 'green','blue','orange','magenta'], ecolor='black', capsize=10)
        plt.ylabel('Probability')
        plt.title('{} classification probability'.format(rr[i]))
        #plt.show()
        plt.tight_layout()
        plt.savefig('Clade_analysis_FD/{}_{}_{}_multi_clades.png'.format(rr[i],b,k),bbox_inches = 'tight')
        plt.close()
        
    #print('Validation Accuracy:'+str(np.mean(Acc))+' +/- '+str(np.std(Acc)))
    
    
            
    #print("Important Features: {}".format(feat_set))
    f.write(str(D)+'\n')
    f.write(str(feat_set_ind)+'\n')
    ct_1 = 0
    ct_2 = 0
    ct_3 = 0
    ct_4 = 0
    ct_5 = 0
    g_ind_20A = []
    g_ind_19A = []
    g_ind_20B = []
    g_ind_19B = []
    g_ind_20C = []
    if len(T) > 0:
        #f.write('Results for Independent Test set: '+str(np.mean(Acc_T))+'+/-'+str(np.std(Acc_T))+'\n')
        probs_mean_T = np.mean(sc_prob_T,axis = 0)
        probs_std_T = np.std(sc_prob_T,axis = 0)
        for j,v in enumerate(scores_T):
            if v == 0:
                g_ind_20A.append(j)
                ct_1+=1
            if v == 1:
                g_ind_19A.append(j)
                ct_2+=1
            if v == 2:
                g_ind_20B.append(j)
                ct_3+=1
            if v == 3:
                g_ind_19B.append(j)
                ct_4+=1
            if v == 4:
                g_ind_20C.append(j)
                ct_5+=1
        #f.write("Percentage of {}: {}".format(rr[0],ct_1/len(scores_T)*100)+'\n')
        #f.write("Percentage of {}: {}".format(rr[1],ct_2/len(scores_T)*100)+'\n')
        #f.write("Percentage of {}: {}".format(rr[2],ct_3/len(scores_T)*100)+'\n')
        #f.write("Percentage of {}: {}".format(rr[3],ct_4/len(scores_T)*100)+'\n')
        #f.write("Percentage of {}: {}".format(rr[4],ct_5/len(scores_T)*100)+'\n')
        plt.bar(rr,probs_mean_T,yerr=probs_std_T, align='center', color=['red', 'green','blue','orange','magenta'], ecolor='black', capsize=10)
        plt.ylabel('Probability')
        plt.title('classification probability')
        #plt.show()
        plt.tight_layout()
        f.write('20A:'+str(g_ind_20A)+'\n')
        f.write('19A:'+str(g_ind_19A)+'\n')
        f.write('20B:'+str(g_ind_20B)+'\n')
        f.write('19B:'+str(g_ind_19B)+'\n')
        f.write('20C:'+str(g_ind_20C)+'\n')
        
    return np.mean(Acc),np.std(Acc)

In [33]:
def ML_different_test(b,k,V,beta,gamma,classes):
    word_list = [''.join(x) for x in itertools.product('acgt', repeat=k)]
    wd_next = [''.join(x) for x in itertools.product('acgt', repeat=b)]
    X = []
    y = []
    ct = 0
    n = len(V)
    G = [[] for i in range(n)]
    
    for seq in V:
        for s in seq:
            F_list,en_list = stats(s,b,k,beta,gamma)
            G[ct]+=F_list
        print(len(G[ct]))
        ct+=1
    
    
    min_val = float('inf')
    for i in range(n-1):
        if min_val > len(G[i]):
            min_val = len(G[i])
    
    for i in range(n-1):
        random.shuffle(G[i])
        X+=[G[i][j] for j in range(len(G[i]))]
        y+=[i for j in range(len(G[i]))]
    for i in range(n-1,n):
        X+=[G[i][j] for j in range(len(G[i]))]
        y+=[i for j in range(len(G[i]))]
    #min_val = int(3*(min_val)/4)   
    print('Length of X: {}'.format(len(X)))
    X_original = [x_i for x_i in X]
    X = np.array(X)
    y = np.array(y)
    X_original = np.array(X_original)
    X = preprocessing.scale(X)
    U_X = list(X[:min_val])
    U_y = list(y[:min_val])
    T_X = list(X[len(G[0])+len(G[1])+len(G[2])+len(G[3])+len(G[4]):])
    T_y = list(y[len(G[0])+len(G[1])+len(G[2])+len(G[3])+len(G[4]):])
    running_sum = len(G[0])
    for i in range(1,n-1):
        U_X+=list(X[running_sum:running_sum+min_val])
        U_y+=list(y[running_sum:running_sum+min_val])
        running_sum+=len(G[i])
    zipped_U = list(zip(U_X,U_y))
    random.shuffle(zipped_U)
    U_X = [x_i for x_i,_ in zipped_U]
    U_y = [y_i for _,y_i in zipped_U]
    zipped_T = list(zip(T_X,T_y))
    #random.shuffle(zipped_T)
    T_X = [x_i for x_i,_ in zipped_T]
    T_y = [y_i for _,y_i in zipped_T]
    U_X = np.array(U_X)
    U_y = np.array(U_y)
    T_X = np.array(T_X)
    T_y = np.array(T_y)
    return U_X,U_y,X_original,T_X,T_y

In [34]:
def ML_same(b,k,V,beta,gamma):
    word_list = [''.join(x) for x in itertools.product('acgt', repeat=k)]
    wd_next = [''.join(x) for x in itertools.product('acgt', repeat=b)]
    X = []
    y = []
    ct = 0
    n = 2
    G = [[] for i in range(n)]
    UU = []
    for seq in V:
        for s in seq:
            F_list,en_list = stats(s,b,k,beta,gamma)
            UU+=F_list
        print(len(UU))
    lx = len(UU)
    ux = int(lx/2)
    random.shuffle(UU)
    G[0]+=[UU[j] for j in range(ux)]
    G[1]+=[UU[j] for j in range(ux,lx)] 
    print(len(G[0]))
    print(len(G[1]))
    min_val = float('inf')
    for i in range(n):
        if min_val > len(G[i]):
            min_val = len(G[i])
            
    for i in range(n):
        random.shuffle(G[i])
        X+=[G[i][j] for j in range(len(G[i]))]
        y+=[i for j in range(len(G[i]))]
    #min_val = int(3*(min_val)/4)   
    print('Length of X: {}'.format(len(X)))
    X_original = [x_i for x_i in X]
    X = np.array(X)
    y = np.array(y)
    X_original = np.array(X_original)
    X = preprocessing.scale(X)
    U_X = list(X[:min_val])
    U_y = list(y[:min_val])
    T_X = list(X[min_val:len(G[0])])
    T_y = list(y[min_val:len(G[0])])
    running_sum = len(G[0])
    for i in range(1,n):
        U_X+=list(X[running_sum:running_sum+min_val])
        U_y+=list(y[running_sum:running_sum+min_val])
        T_X+=list(X[running_sum+min_val:running_sum+len(G[i])])
        T_y+=list(y[running_sum+min_val:running_sum+len(G[i])])
        running_sum+=len(G[i])
    zipped_U = list(zip(U_X,U_y))
    random.shuffle(zipped_U)
    U_X = [x_i for x_i,_ in zipped_U]
    U_y = [y_i for _,y_i in zipped_U]
    zipped_T = list(zip(T_X,T_y))
    random.shuffle(zipped_T)
    T_X = [x_i for x_i,_ in zipped_T]
    T_y = [y_i for _,y_i in zipped_T]
    U_X = np.array(U_X)
    U_y = np.array(U_y)
    T_X = np.array(T_X)
    T_y = np.array(T_y)
    return U_X,U_y,X_original,T_X,T_y

In [ ]:
l = [[1,4]]
beta = 0.5
gamma = 0.5
clades = ['20A','19A','20B','19B','20C']
folder = 'Clade_Analysis/Multiclassifier'
cutoff = -10
plt.rcParams.update({'font.size': 14})

with open('Clade_Analysis/V_clades.pkl','rb') as g:  
    V = pickle.load(g)
f = open('Clade_Analysis/Results.txt','w')
for val in l:
    b,k = val[0],val[1]
    X,y,X_original,T,y_T = ML(b,k,V,beta,gamma,clades)
    mean,std = build_model(X,y,X_original,b,k,clades,cutoff,T,y_T,f)

f.close()